In [37]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from config import password

In [31]:
file = "aqi_full_data.csv"
air_quality_df = pd.read_csv(file, index_col=0)

file2 = "ca_sites.csv"
sites_df = pd.read_csv(file2, index_col=0)

file3 = "census_info.csv"
census_df = pd.read_csv(file3, index_col=0)

In [36]:
air_quality_df.dtypes

State Name            object
county Name           object
Date                  object
AQI                    int64
Category              object
Defining Parameter    object
Defining Site         object
dtype: object

In [34]:
sites_df = sites_df.rename_axis('site_no')
site_count = sites_df.groupby('site_no')
site_unique = site_count['County Name'].unique().count()
print(site_unique)
sites_df.head()

1017


,Latitude,Longitude,Elevation,Land Use,Location Setting,State Name,County Name,County Code,City Name,CBSA Name
site_no,,,,,,,,,,
06-001-0001,37.876870,-122.266913,15.0,RESIDENTIAL,URBAN AND CENTER CITY,California,Alameda,1,Berkeley,"San Francisco-Oakland-Hayward, CA"
06-001-0002,37.533243,-121.958813,22.0,COMMERCIAL,URBAN AND CENTER CITY,California,Alameda,1,Fremont,"San Francisco-Oakland-Hayward, CA"
06-001-0003,37.684900,-121.765900,150.0,COMMERCIAL,URBAN AND CENTER CITY,California,Alameda,1,Livermore,"San Francisco-Oakland-Hayward, CA"
06-001-0004,37.800484,-122.266358,38.0,COMMERCIAL,URBAN AND CENTER CITY,California,Alameda,1,Oakland,"San Francisco-Oakland-Hayward, CA"
06-001-0005,37.798600,-122.268400,10.0,COMMERCIAL,URBAN AND CENTER CITY,California,Alameda,1,Oakland,"San Francisco-Oakland-Hayward, CA"


In [5]:
census_df = census_df.dropna().apply(lambda x: x.str.strip() if x.dtype == "object" else x)
census_count = census_df.dropna().groupby('County')

census_unique = census_count['County'].unique().count()
census_df.head()

,County,Census,Estimates Base,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
1,Alameda,1510271,1510258,1512986,1530915,1553764,1579593,1607792,1634538,1650950,1660196,1666756,1671329
2,Alpine,1175,1175,1161,1093,1110,1128,1080,1077,1047,1111,1089,1129
3,Amador,38091,38091,37886,37543,37104,36620,36726,37031,37429,38529,39405,39752
4,Butte,220000,220005,219949,219975,220869,221641,223516,224631,226231,228696,230339,219186
5,Calaveras,45578,45578,45468,45160,44815,44655,44671,44965,45322,45681,45698,45905


In [6]:
county_name = sites_df.groupby('County Code')
county_name_unique = county_name['County Name'].first()
county_code_unique = county_name['County Name'].unique().count()

county_df = pd.DataFrame({'county_name':county_name_unique}).rename_axis('country_code')
county_df.head()

,County
County Code,
1,Alameda
3,Alpine
5,Amador
7,Butte
9,Calaveras


In [8]:
county_code_census = county_df.reset_index().merge(census_df,on='County')
county_code_census.head()

,County Code,County,Census,Estimates Base,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,1,Alameda,1510271,1510258,1512986,1530915,1553764,1579593,1607792,1634538,1650950,1660196,1666756,1671329
1,3,Alpine,1175,1175,1161,1093,1110,1128,1080,1077,1047,1111,1089,1129
2,5,Amador,38091,38091,37886,37543,37104,36620,36726,37031,37429,38529,39405,39752
3,7,Butte,220000,220005,219949,219975,220869,221641,223516,224631,226231,228696,230339,219186
4,9,Calaveras,45578,45578,45468,45160,44815,44655,44671,44965,45322,45681,45698,45905


In [9]:
county_code_census = county_code_census.drop(columns=['County', 'Census','Estimates Base']).set_index('County Code')
county_code_census.head()

,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
County Code,,,,,,,,,,
1,1512986,1530915,1553764,1579593,1607792,1634538,1650950,1660196,1666756,1671329
3,1161,1093,1110,1128,1080,1077,1047,1111,1089,1129
5,37886,37543,37104,36620,36726,37031,37429,38529,39405,39752
7,219949,219975,220869,221641,223516,224631,226231,228696,230339,219186
9,45468,45160,44815,44655,44671,44965,45322,45681,45698,45905


In [38]:
county_code_census_pop_year = county_code_census.stack().reset_index().rename(columns={'County Code':'county_code','level_1':'year',0:'population'})
county_code_census_pop_year

,county_code,year,population
0,1,2010,1512986
1,1,2011,1530915
2,1,2012,1553764
3,1,2013,1579593
4,1,2014,1607792
...,...,...,...
575,115,2015,74045
576,115,2016,74952
577,115,2017,76578
578,115,2018,77557


In [ ]:
# create connection to ETL_project_DB in postgres
engine = create_engine('postgresql://postgres:'+ password + '@localhost:5432/AirQuality')
connection = engine.connect()

In [ ]:
# check table names in database
engine.table_names()

In [ ]:
# load dataframes into postgres using pandas
county_df.to_sql(name='Country', con=engine, if_exists='append', index=False)

county_code_census_pop_year.to_sql(name='CensusPopulation', con=engine, if_exists='append', index=False)

sites_df.to_sql(name='Sites', con=engine, if_exists='append', index=False)

air_quality_df.to_sql(name='AirQuality', con=engine, if_exists='append', index=False)
